In [1]:
import pandas as pd
import autograd.numpy as np

from envelope_class import *

from joblib import Parallel, delayed
import multiprocessing

import warnings
warnings.filterwarnings("ignore")

In [2]:
# # 10*10
# si_10 = np.array([[(i+0.5)/10,(j+0.5)/10] for i in range(10) for j in range(10)])
# # 15*15 
# si_15 = np.array([[(i+0.5)/15,(j+0.5)/15] for i in range(15) for j in range(15)])
# # 20*20
# si_20 = np.array([[(i+0.5)/20,(j+0.5)/20] for i in range(20) for j in range(20)])

# si_rand10 = np.array([(randint(0,10)/10,randint(0,10)/10) for i in range(10**2)])
# si_rand15 = np.array([(randint(0,10)/10,randint(0,10)/10) for i in range(15**2)])
# si_rand20 = np.array([(randint(0,10)/10,randint(0,10)/10) for i in range(20**2)])

# si_list = [si_10,si_15,si_20,si_rand10,si_rand15,si_rand20]



In [3]:
# si_4 = np.array([[(i+0.5)/4,(j+0.5)/4] for i in range(4) for j in range(4)])
# si = si_4
# theta = np.array([0.5,1])
# X_list, Y_list = generate(si,theta,seed = 2020)
# thershold = 1e-5
# u = 2

In [4]:
# err_raw_lr = list()
# err_raw_env = list()
# err_raw_spc_env = list()
# # for index in range(6,7):
# for index in range(16):
#     print("Start for {}th project".format(index))
#     X_train,Y_train = np.delete(X_list,index,0),np.delete(Y_list,index,0)
#     si_train = np.delete(si,index,0)
#     X_pred,Y_pred = X_list[index].reshape(1,6),Y_list[index].reshape(1,5)
#     err_lr = LinearRegression().fit(X_train,Y_train).predict(X_pred) - Y_pred
#     err_raw_lr.append(err_lr)

#     alpha_env,beta_env = envelope(X_train,Y_train,u)
#     err_env = alpha_env + np.matmul(X_pred,beta_env) - Y_pred
#     err_raw_env.append(err_env)

#     alpha_spc_env,beta_spc_env = spatial_envelope(X_train,Y_train,si_train,theta,u,thershold)
#     err_spc_env = alpha_spc_env + np.matmul(X_pred,beta_spc_env) - Y_pred 
#     err_raw_spc_env.append(err_spc_env)

In [5]:
# n = len(X_list)
# err_lr_final = sum([np.sum(err_i**2) for err_i in err_raw_lr])/n
# err_env_final = sum([np.sum(err_i**2) for err_i in err_raw_env])/n
# err_spc_env_final = sum([np.sum(err_i**2) for err_i in err_raw_spc_env])/n

In [6]:
# err_lr_final,err_env_final,err_spc_env_final

## Total Cases!
1. $\{(X_i,Y_i)\}$ err generation methods $*$ 3
2. fitting methods already included
3. data numbers $*$ 3

In [7]:
def spatial_envelope(X_env,Y_env,si,theta,u,thershold):
    #const:
    #n,p,r
    #H,G
    #beta_MLE
    #si: matrix of loacation
    n,p,r = X_env.shape[0],X_env.shape[1],Y_env.shape[1]
    H = Y_env - np.kron(np.mean(Y_env,axis = 0).reshape(1,r),np.repeat(1,n).reshape(n,1))
    G = X_env - np.kron(np.mean(X_env,axis = 0).reshape(1,p),np.repeat(1,n).reshape(n,1))
    
    linear_model = LinearRegression().fit(X_env,Y_env)
    err = Y_env - linear_model.predict(X_env)
    beta_MLE = linear_model.coef_
    print("Start")
    # changable
    # judge, iter_count, err_count, theta
    # V0_hat, V1_hat, PV0_hat, PV1_hat
    # Sigma_Y, Sigma_res
    # rho_h_theta
    Sigma_res = np.cov(err.T)
    Sigma_Y = np.cov(Y_env.T)

    judge = True
    iter_count = 0
    cal_thershold,min_thershold = thershold,thershold
    prot = 1
    err_count = 0
    while judge:
        try:
            if err_count > 10:
                print("Start new")
                theta = np.random.uniform(0,0.5,2)
                err_count = 0
            # Step1: Optimize on Gamma to get V0,V1,PV0,PV1
            def cost(Gamma):
                X = np.matmul(Gamma,Gamma.T)
                out = -np.log(np.linalg.det(np.matmul(np.matmul(X,Sigma_res),X) + np.matmul(np.matmul(np.eye(r) - X,Sigma_Y),np.eye(r) - X)))
                return(np.array(out))

            manifold = Grassmann(r,u)
            #manifold = Stiefel(r,u)
            problem = Problem(manifold=manifold, cost=cost,verbosity=0)
            solver = SteepestDescent()
            Gamma = solver.solve(problem)
            PV1_hat = np.matmul(Gamma,Gamma.T)
            PV0_hat = np.eye(r) - PV1_hat

            V1_hat = np.matmul(np.matmul(PV1_hat,Sigma_Y),PV1_hat)
            V0_hat = np.matmul(np.matmul(PV1_hat,Sigma_res),PV1_hat)    
            # Step2: Optimize on theta
            def theta_fun(theta):
                rho_h_theta= np.array(rho(si,theta))
                item1 = np.matmul(sqrtm(np.linalg.inv(rho_h_theta).real).real,G)
                project = lambda x: np.eye(n) - np.matmul(np.matmul(x,np.linalg.inv(np.matmul(x.T,x)).real),x.T)
                item2 = np.matmul(np.matmul(project(item1),sqrtm(np.linalg.inv(rho_h_theta).real).real),H)
                item3 = np.matmul(np.matmul(item2,np.linalg.pinv(V1_hat).real),item2.T)
                item4 = np.matmul(sqrtm(np.linalg.inv(rho_h_theta).real).real,H)
                item5 = np.matmul(np.matmul(item4,np.linalg.pinv(V0_hat).real),item4.T)
                loss = r*np.linalg.det(rho_h_theta) + 0.5 * np.trace(item3 + item5)
                return(loss)
#             print("Theta: {}".format(theta))
            opt_res = minimize(theta_fun,theta,method="BFGS")
#             print("Pass")
            weight = max(min(1,1/cal_thershold),(thershold/prot)**(1 - 1/(iter_count+1)))
            theta_opt = np.abs(np.array(opt_res.x))
            theta_new = (1 - weight) * theta + weight * theta_opt
            theta = theta_new
#             theta = np.array(opt_res.x)
            # Step3 update Sigma_Y, Sigma_Res based on theta
            rho_h_theta= np.array(rho(si,theta))
            term1 = np.matmul(np.matmul(H.T,np.linalg.inv(rho_h_theta).real),H)
            term2 = np.matmul(np.matmul(G.T,np.linalg.inv(rho_h_theta).real),H)
            term3 = np.matmul(np.matmul(G.T,np.linalg.inv(rho_h_theta).real),G)

            Sigma_Y = term1
            Sigma_res = term1 - np.matmul(np.matmul(term2.T,np.linalg.inv(term3).real),term2)

            if iter_count == 0:
                iter_count += 1
                oldV0_hat,oldV1_hat,old_theta = V0_hat,V1_hat,theta
                continue
#             print("Before thershold")
            cal_thershold = np.sum((oldV1_hat - V1_hat)**2) + np.sum((oldV0_hat - V0_hat)**2) + np.sum((old_theta - theta)**2)
            print("Gap: {}, Theta: {}, weight: {}".format(cal_thershold,theta,weight))
            if cal_thershold < thershold:
                judge = False
                min_thershold = min(min_thershold,cal_thershold)
                prot = cal_thershold/min_thershold

            oldV0_hat,oldV1_hat,old_theta = V0_hat,V1_hat,theta
            iter_count += 1
        except:
            err_count += 1
            theta = theta + np.array([randint(-10,10)*thershold*prot,randint(-10,10)*thershold*prot])
            X_env = X_env + np.random.normal(0,1e-6,n*p).reshape(n,p)
            Y_env = Y_env + np.random.normal(0,1e-6,n*r).reshape(n,r)
            continue
            
    beta_final = np.matmul(PV1_hat,beta_MLE)
    Y_bar = np.mean(Y_env,axis = 0)
    X_bar = np.mean(X_env,axis = 0)
    alpha_final = Y_bar - np.matmul(X_bar,beta_final.T)
    output = (alpha_final.reshape(1,r),beta_final.reshape(p,r))
    print("stop, iter = {}".format(iter_count+err_count))
        
    return(output)

In [8]:
si_10 = np.array([[(i+0.5)/10,(j+0.5)/10] for i in range(10) for j in range(10)])
si_15 = np.array([[(i+0.5)/15,(j+0.5)/15] for i in range(15) for j in range(15)])
si_20 = np.array([[(i+0.5)/20,(j+0.5)/20] for i in range(20) for j in range(20)])

si_rand10 = np.array([(randint(0,10)/10,randint(0,10)/10) for i in range(10**2)])
si_rand15 = np.array([(randint(0,10)/10,randint(0,10)/10) for i in range(15**2)])
si_rand20 = np.array([(randint(0,10)/10,randint(0,10)/10) for i in range(20**2)])

si_dict = {"si_10":si_10,"si_15":si_15,"si_20":si_20,"si_10_rand":si_rand10,"si_15_rand":si_rand15,"si_20_rand":si_rand20}

theta_1st = np.array([0,0])
theta_2nd = np.array([1,0.5])
theta_3rd = np.array([1,0.5])

theta_dict = {"case1":theta_1st,"case2":theta_2nd,"case3":theta_3rd}

In [9]:
def run_simulation(si_name,theta_name):
    si,theta = si_dict[si_name],theta_dict[theta_name]
    
    X_list,Y_list = generate(si,theta)
    thershold = 1e-5
    u = 2
    case_length = len(si)
    
    err_raw_lr = list()
    err_raw_env = list()
    err_raw_spc_env = list()
    # for index in range(6,7):
    for index in range(case_length):
        print("Start for {}th project".format(index))
        X_train,Y_train = np.delete(X_list,index,0),np.delete(Y_list,index,0)
        si_train = np.delete(si,index,0)
        X_pred,Y_pred = X_list[index].reshape(1,6),Y_list[index].reshape(1,5)
        err_lr = LinearRegression().fit(X_train,Y_train).predict(X_pred) - Y_pred
        err_raw_lr.append(err_lr)

        alpha_env,beta_env = envelope(X_train,Y_train,u)
        err_env = alpha_env + np.matmul(X_pred,beta_env) - Y_pred
        err_raw_env.append(err_env)

        theta_use = theta + np.random.uniform(0,1,2)
        alpha_spc_env,beta_spc_env = spatial_envelope(X_train,Y_train,si_train,theta_use,u,thershold)
        err_spc_env = alpha_spc_env + np.matmul(X_pred,beta_spc_env) - Y_pred 
        err_raw_spc_env.append(err_spc_env)
    
    err_lr_final = sum([np.sum(err_i**2) for err_i in err_raw_lr])/case_length
    err_env_final = sum([np.sum(err_i**2) for err_i in err_raw_env])/case_length
    err_spc_env_final = sum([np.sum(err_i**2) for err_i in err_raw_spc_env])/case_length
    
    return(si_name,theta_name,err_lr_final, err_env_final, err_spc_env_final)

In [10]:
# num_cores = multiprocessing.cpu_count()
# res = Parallel(n_jobs=num_cores)(delayed(run_simulation)(si,theta) for si,theta in si_theta_name_list)


In [11]:
res = run_simulation("si_10","case1")

Start for 0th project
Start
Gap: 2402002.6203892627, Theta: [1.44293411e+03 3.49342253e-03], weight: 1
Gap: 2048.7245546410263, Theta: [1.44293411e+03 3.49344169e-03], weight: 0.0004641588833612775
Gap: 5.132830877560431e-07, Theta: [1.44293411e+03 3.49346182e-03], weight: 0.00048810856380603984
stop, iter = 4
Start for 1th project
Start
Gap: 334798.4966336301, Theta: [0.00480629 2.00265892], weight: 1
Gap: 4.865085440496207e-06, Theta: [0.00480629 2.00265892], weight: 0.0004641588833612775
stop, iter = 3
Start for 2th project
Start
Gap: 888407.7942878256, Theta: [7.39084023e+02 3.75924317e-03], weight: 1
Gap: 1726.7632898224865, Theta: [7.39084023e+02 3.75925827e-03], weight: 0.0004641588833612775
Gap: 0.0002902240048737457, Theta: [7.39084023e+02 3.75927710e-03], weight: 0.0005791181720702443
Gap: 0.0003511444404712327, Theta: [7.39084031e+02 3.79177106e-03], weight: 1
Gap: 0.175804127073048, Theta: [7.39084031e+02 3.79215649e-03], weight: 1
Gap: 0.00035558174637187836, Theta: [7.390

In [13]:
res

('si_10', 'case1', 15.558158191357046, 14.610784414933692, 14.598959784413802)

In [16]:
out_df = pd.DataFrame(res).transpose()
out_df.columns = ["si","case","LR","ENV","SPEC_ENV"]
out_df.to_csv("res.csv")